# Extracting sign distributions

This notbook contains code for:
- importing necessary libraries
- loading stored WTs
- creating random sparse networks
- Extracting sign distributions from WTs
- Extracting sign distributions from non-WTs
- saving extracted distributions in files

### Requirements:

In [1]:
# importing necessary libraries and the cnn architecture I defined

from cnn_architecture import CNN2Model
from utils import *
from load_datasets import load_and_prep_dataset

import tensorflow_datasets as tfds
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import random

2024-05-30 16:38:04.064045: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
# extract sign distribution from one layer

def sign_distribution_layer(this_layer, next_layer):
    '''
    columns: 
    ["prune_rate_in", "prune_rate_out", "pos_in", "pos_out", "neg_in", "neg_out", "sign_rate_in", "sign_rate_out"]
    '''
    sign_distr = pd.DataFrame()
    unconnected_neurons = {"no_incomming": 0 , "no_outgoing" : 0, "no_total_conncetions" : 0}
    for this_neuron, next_neurons, i in zip(this_layer.T, next_layer, range(np.shape(this_layer)[0])):
        
        this_unpruned = this_neuron[this_neuron != 0.0]
        next_unpruned = next_neurons[next_neurons != 0.0]
        
        # only include neurons that receive and propagate input
        if len(this_unpruned)*len(next_unpruned) != 0:
            
            dic = {} 
            dic["pos_in"] = len(this_unpruned[this_unpruned>0])
            dic["pos_out"] = len(next_unpruned[next_unpruned>0])
            dic["neg_in"] = len(this_unpruned[this_unpruned<0])
            dic["neg_out"] = len(next_unpruned[next_unpruned<0])
            dic["prune_rate_in"] = 1 - (len(this_unpruned)/len(this_neuron))
            dic["prune_rate_out"] = 1 - (len(next_unpruned)/len(next_neurons))
            dic["sign_rate_in"] = dic["pos_in"]/len(this_unpruned)   
            dic["sign_rate_out"] = dic["pos_out"]/len(next_unpruned)                               

            df = pd.DataFrame(data = dic, index = [i])
            sign_distr = pd.concat([sign_distr, df], axis=0)
        
        # store information about inconnected neurons
        else :
            
            if len(this_unpruned) == 0:
                if len(next_unpruned) ==0:
                    unconnected_neurons["no_total_conncetions"] = unconnected_neurons["no_total_conncetions"] + 1
                else:
                    unconnected_neurons["no_incomming"] = unconnected_neurons["no_incomming"] + 1
            else:
                unconnected_neurons["no_outgoing"] = unconnected_neurons["no_outgoing"] + 1
        
    return sign_distr, unconnected_neurons

In [6]:
# extract sign distribution form two layers

def sign_distribution_layers(layer1, layer2, layer3):
    
    sign_distr1, unconnected_neurons1 = sign_distribution_layer(layer1, layer2)
    sign_distr2, unconnected_neurons2 = sign_distribution_layer(layer2, layer3)
    sign_distr1["layer"] = "dense1"
    sign_distr2["layer"] = "dense2"
    sign_distr_both = pd.concat([sign_distr1, sign_distr2], axis=0)
    
    unconnected_neurons1["layer"] = "dense1"
    unconnected_neurons2["layer"] = "dense2"

    return sign_distr_both, unconnected_neurons1, unconnected_neurons2

## Extracting sign distributions and storing them as files:

In [5]:
wts_per_dataset = 15

In [6]:
# extract sign distributions

def get_sign_distr(dataset, pruning_name, n):

    # collect statistics of unconnected neurons
    unconnected_statistics = pd.DataFrame()
    
    # make a model to load the weights into
    train_dataset, test_dataset = load_and_prep_dataset(dataset, batch_size=60, shuffle_size=512)
    model = CNN2Model()
    model(list(train_dataset)[0][0])
    for i in tqdm(range(n), leave=False, desc=f"sign_distributions of {dataset}"):
        
        # get WT weights
        model.load_weights(f"1b WTs/WT_{dataset}_{pruning_name}_{i}")    
        weights_wt = model.get_weights()
        # get sign distribution
        sign_distr_wt, unconnected_neurons1, unconnected_neurons2 = sign_distribution_layers(weights_wt[4], weights_wt[6], weights_wt[8])
        # store sign distribution
        sign_distr_wt.to_csv(f'2b Sign distributions/{dataset}_{pruning_name}_{i}_sign_distr.csv', index=False)
        
        
        #collect unconnected neurons statistics
        unconnected_neurons1["model"] = f"WT_{dataset}_{pruning_name}_{i}"
        unconnected_neurons2["model"] = f"WT_{dataset}_{pruning_name}_{i}"
        df1 = pd.DataFrame(data = unconnected_neurons1, index = [i])
        df2 = pd.DataFrame(data = unconnected_neurons2, index = [i])
        unconnected_statistics = pd.concat([unconnected_statistics, df1, df2], axis=0)
        
    unconnected_statistics.to_csv(f'2b Sign distributions/{dataset}_{pruning_name}_unconnected_statistic.csv', index=False)

In [7]:
# store sign distributions of Wts

#for dataset in ["SVHN","CINIC","CIFAR"]:
#    get_sign_distr(dataset,"IMP",wts_per_dataset)

### Generating randomly pruned sign distributions

In [8]:
# implement random pruning of a weight matrix according to a pruning rate

def random_pruning(model,pruning_rates):
    
    weights = model.get_weights()
    new_weights = []
    
    for layer, p_rate in zip(weights[::2], pruning_rates):
        shape = np.shape(layer)
        layer = layer.flatten()
        number_to_prune = int(len(layer) * p_rate)
        pruning_indexi = random.sample(range(len(layer)),number_to_prune)
        layer[pruning_indexi] = 0.0
        layer = np.reshape(layer,shape)
        new_weights.append(layer)
        
    weights[::2] = new_weights
    model.set_weights(weights)
    weights = model.get_weights()
    
    return model

In [9]:
def make_random_sign_distr(n):

    # get pruning rates from imp model
    train_dataset, test_dataset = load_and_prep_cifar(batch_size=60, shuffle_size=512)
    model_cifar_imp = CNN2Model()
    model_cifar_imp(list(train_dataset)[0][0])
    model_cifar_imp.load_weights("1b WTs/WT_CIFAR_IMP_0")
    pruning_rates = get_pruning_rates(model_cifar_imp.get_weights()[::2])

    unconnected_statistics = pd.DataFrame()

    # generate random sparse networks
    for i in range(n):
        
        # randomly prune a model
        random_model = CNN2Model()
        random_model(list(train_dataset)[0][0])
        random_model = random_pruning(random_model,pruning_rates)
        weights_rsn = random_model.get_weights()
        
        # store sign distributions of random sparse networks
        sign_distr_random, unconnected_neurons1, unconnected_neurons2  = sign_distribution_layers(weights_rsn[4], weights_rsn[6], weights_rsn[8])
        sign_distr_random.to_csv(f"2b Sign distributions/RSN_{i}_sign_distr.csv", index=False)
        
        #collect unconnected neurons statistics
        unconnected_neurons1["model"] = f"RSN_{i}"
        unconnected_neurons2["model"] = f"RSN_{i}"
        df1 = pd.DataFrame(data = unconnected_neurons1, index = [i])
        df2 = pd.DataFrame(data = unconnected_neurons2, index = [i])
        unconnected_statistics = pd.concat([unconnected_statistics, df1, df2], axis=0)

    # store unconnected statistics
    unconnected_statistics.to_csv(f'2b Sign distributions/RSN_unconnected_statistic.csv', index=False)

In [ ]:
#make_random_sign_distr(wts_per_dataset)

## Get Pruning rates

In [3]:
train_dataset, test_dataset = load_and_prep_dataset("CIFAR", batch_size=60, shuffle_size=512)
model_cifar_imp = CNN2Model()
model_cifar_imp(list(train_dataset)[0][0])
model_cifar_imp.load_weights("1b WTs/WT_CIFAR_IMP_0")

2024-05-29 22:18:42.192152: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-29 22:18:42.246078: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-29 22:18:42.246369: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

ValueError: File format not supported: filepath=1b WTs/WT_CIFAR_IMP_0. Keras 3 only supports V3 `.keras` and `.weights.h5` files, or legacy V1/V2 `.h5` files.

In [14]:
# return the pruning rates of the conv and dense layers
print("conv rate (without biases): ", get_pruning_rate(model_cifar_imp.get_weights()[0:4:2]))
print("dense rate (without biases): ", get_pruning_rate(model_cifar_imp.get_weights()[4::2]))
print("conv rate (with biases): ", get_pruning_rate(model_cifar_imp.get_weights()[0:4:]))
print("dense rate (with biases): ", get_pruning_rate(model_cifar_imp.get_weights()[4::]))
print("pruning rates without biases each layer: ", get_pruning_rates(model_cifar_imp.get_weights()[::2]))
print("pruning rates with biases each layer: ")
print(get_pruning_rate(model_cifar_imp.get_weights()[[0,1]]))
print(get_pruning_rate(model_cifar_imp.get_weights()[[2,3]]))
print(get_pruning_rate(model_cifar_imp.get_weights()[[4,5]]))
print(get_pruning_rate(model_cifar_imp.get_weights()[[6,7]]))
print(get_pruning_rate(model_cifar_imp.get_weights()[[8,9]]))

print("pruning rate total with biases: ", get_pruning_rate(model_cifar_imp.get_weights()))
print("pruning rate total without biases: ", get_pruning_rate(model_cifar_imp.get_weights()[::2]))

conv rate (without biases):  0.6794931592039801
dense rate (without biases):  0.9176456925675676
conv rate (with biases):  0.6772469008264463
dense rate (with biases):  0.9175333257329128
pruning rates without biases each layer:  [0.6799768518518519, 0.6794704861111112, 0.9284100532531738, 0.259185791015625, 0.13789062500000004]
pruning rate total with biases:  0.9153704562118372
pruning rate total without biases:  0.9155087942502567


## Get sign distributions for different pruning rates

In [7]:
def get_sign_distr(dataset, p_rate, n):

    # collect statistics of unconnected neurons
    unconnected_statistics = pd.DataFrame()
    
    # make a model to load the weights into
    train_dataset, test_dataset = load_and_prep_dataset("CIFAR", batch_size=60, shuffle_size=512)
    model = CNN2Model()
    model(list(train_dataset)[0][0])
    for i in tqdm(range(n), leave=False, desc=f"sign_distributions of {dataset}"):
        
        # get WT weights
        model.load_weights(f"1b WTs/more_p_rates/WT_{dataset}_IMP{p_rate}_{i}.weights.h5")    
        weights_wt = model.get_weights()
        # get sign distribution
        sign_distr_wt, unconnected_neurons1, unconnected_neurons2 = sign_distribution_layers(weights_wt[4], weights_wt[6], weights_wt[8])
        # store sign distribution
        sign_distr_wt.to_csv(f'2b Sign distributions/{dataset}_IMP{p_rate}_{i}_sign_distr.csv', index=False)
        
        
        #collect unconnected neurons statistics
        unconnected_neurons1["model"] = f"WT_{dataset}_IMP{p_rate}_{i}"
        unconnected_neurons2["model"] = f"WT_{dataset}_IMP{p_rate}_{i}"
        df1 = pd.DataFrame(data = unconnected_neurons1, index = [i])
        df2 = pd.DataFrame(data = unconnected_neurons2, index = [i])
        unconnected_statistics = pd.concat([unconnected_statistics, df1, df2], axis=0)
        
    unconnected_statistics.to_csv(f'2b Sign distributions/more_p_rates/{dataset}_IMP{p_rate}_unconnected_statistic.csv', index=False)

In [3]:
train_dataset, test_dataset = load_and_prep_dataset("CIFAR", batch_size=60, shuffle_size=512)
model = CNN2Model()
model(list(train_dataset)[0][0])
model.load_weights(f"1b WTs/more_p_rates/WT_CIFAR_IMP30_0.weights.h5")
print(get_pruning_rates(model.get_weights()))

/net/projects/scratch/summer/valid_until_31_January_2025/epetersen/miniconda3/envs/thesis/lib/python3.11/site-packages/keras/src/layers/pooling/base_pooling.py:23: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(name=name, **kwargs)
2024-05-30 16:39:25.809411: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


[0.1504629629629629, 0.0, 0.15001085069444442, 0.0, 0.3039360046386719, 0.0, 0.058380126953125, 0.0, 0.037109375, 0.0]


In [1]:
p_rates = [30,51,65,76,80,83,88,92,94,96,97]
for p_rate in p_rates:
    get_sign_distr("CIFAR",p_rate,n=10)

NameError: name 'get_sign_distr' is not defined